In [2]:
import os
import numpy as np
import cv2
import fnmatch
from matplotlib import pyplot as plt

In [8]:
def tils_density(input_path):
    tils_hm_files = fnmatch.filter(os.listdir(input_path+'pred_tils/'), '*Tumor*_gray.png')
    tils_hm_files.sort()
    M=17; N=17
    with open('result_Tumor_tils_density_test.csv', 'a') as outfile :
        outfile.write('file_name,immune_infl,immune_excl,immune_des\n')
        for tils_file in tils_hm_files :
            tissue_mask_file = tils_file.split('_gray')[0] + '.png'
            tumor_file = tils_file
            til = input_path + '/pred_tils/' + tils_file
            tissue_mask = input_path + '/tissue_mask/'+ tissue_mask_file
            tumor = input_path + '/pred_tumor/' + tumor_file
            stroma=input_path + '/pred_stromal/' + tils_file
            til_img = plt.imread(til)
            #plt.imshow(til_img)
            #plt.show()
            #print('til:',til_img.shape)
            R, G, B = til_img[:,:,0], til_img[:,:,1], til_img[:,:,2]
            til_img_gray=np.mean([R,G,B], axis=0)
            til_img_gray[til_img_gray >= 0.5] = 1
            til_img_gray[til_img_gray < 0.5] = 0
            tumor_img = plt.imread(tumor)
            #print('Tumor:',tumor_img.shape)
            #plt.imshow(tumor_img)
            #plt.show()
            R, G, B = tumor_img[:,:,0], tumor_img[:,:,1], tumor_img[:,:,2]
            tumor_img_gray = np.mean([R,G,B], axis=0)
            tumor_img_gray[tumor_img_gray >= 0.5] = 1
            tumor_img_gray[tumor_img_gray < 0.5] = 0
            tumor_img_gray = cv2.resize(tumor_img_gray, (til_img_gray.shape[0],til_img_gray.shape[1]))
            tissue_mask_img = plt.imread(tissue_mask)
            #print('tissue:',tissue_mask_img.shape)
            #plt.imshow( tissue_mask_img)
            #plt.show()
            tissue_mask_img = cv2.resize(tissue_mask_img, (til_img.shape[0],til_img.shape[1], 3))
            R, G, B = tissue_mask_img[:,:,0], tissue_mask_img[:,:,1], tissue_mask_img[:,:,2]
            tissue_mask_img_gray = np.mean([R,G,B], axis=0)
            tissue_mask_img_gray[tissue_mask_img_gray >= 0.5] = 1
            tissue_mask_img_gray[tissue_mask_img_gray < 0.5] = 0
            tiles = [tissue_mask_img_gray[x:x+M,y:y+N] for x in range(0,tissue_mask_img_gray.shape[0],M) for y in range(0,tissue_mask_img_gray.shape[1],N)]
            #print(tiles)
            stroma_img = plt.imread(stroma)
            stroma_img = cv2.resize(stroma_img, (til_img_gray.shape[0],til_img_gray.shape[1]))
            R, G, B = stroma_img[:,:,0], stroma_img[:,:,1], stroma_img[:,:,2]
            stroma_img = np.mean([R,G,B], axis=0)
            #print('stroma:',stroma_img.shape)
            #plt.imshow(stroma_img)
            #plt.show()
            overlap_tt = til_img_gray + tumor_img_gray #overlap TIL and Tumor
            overlap_ts=til_img_gray + stroma_img#overlap of til and stroma
            overlap_tst=tissue_mask_img_gray + til_img_gray
            c_overlap_tt = overlap_tt > 1 #binarize
            c_overlap_ts=overlap_ts > 1
            c_overlap_tst=overlap_tst > 1
            imm_inf=0
            imm_excl=0
            imm_des=0
            c_tissue=0
            i=0
            for x in range(0,tissue_mask_img_gray.shape[0],M):
                for y in range(0,tissue_mask_img_gray.shape[1],N):
                    #print(i)
                    #print(tiles[i].sum())
                    if (tiles[i].sum())>100:
                        tissue_sum=tiles[i].sum()
                        c_tissue+=1 #count the number of tissue tiles
                        overlap_count_tt = sum(map(lambda x : x == True, c_overlap_tt[x:x+M,y:y+N]))#count the overlap ratio
                        overlap_count_ts = sum(map(lambda x : x == True, c_overlap_ts[x:x+M,y:y+N]))
                        overlap_count_tst = sum(map(lambda x : x == True, c_overlap_tst[x:x+M,y:y+N]))
                        sum_overlap_tt = np.sum(overlap_count_tt)
                        sum_overlap_ts = np.sum(overlap_count_ts)
                        sum_overlap_tst = np.sum(overlap_count_tst)
                        #print(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]),sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]),sum_overlap_tst)
                        if np.isnan(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]))==False and sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]) > 0.4:
                            imm_inf+=1
                        elif np.isnan(sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]))==False and sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]) >0.2:
                            imm_excl+=1
                        else:
                            imm_des+=1
                    i+=1
            print(tissue_mask_file.split('_')[0],imm_inf,imm_excl,c_tissue,imm_des)
            outfile.write(tissue_mask_file.split('_')[0]
                              + ',' + str(imm_inf/c_tissue)
                              + ',' + str(imm_excl/c_tissue)
                              + ',' + str(imm_des/c_tissue) + '\n')
    outfile.close()


input_path = "/home/ext_jang_inyeop_mayo_edu/hwangt-share/post_process/VanAbel_Oropharynx_Tumor_TIL_Stroma/Van Abel-K_req32469_OPX TILs HE/"
tils_density(input_path)


error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - Can't parse 'dsize'. Expected sequence length 2, got 3
>  - Can't parse 'dsize'. Expected sequence length 2, got 3
